# Eigenvalue Analysis

Eigenvalue analysis (also called small-signal stability analysis) examines how a power system responds to small perturbations around its operating point. By linearizing the nonlinear differential-algebraic equations and computing the eigenvalues of the resulting state matrix, we can determine whether the system is stable and identify the oscillatory modes that dominate its dynamic behavior.

This analysis is fundamental to understanding inter-area oscillations, local plant modes, and control interactions. The location of eigenvalues in the complex plane tells us about stability: eigenvalues with negative real parts indicate stable modes (perturbations decay), while eigenvalues with positive real parts indicate unstable modes (perturbations grow). The imaginary part relates to the oscillation frequency.

This tutorial demonstrates how to run eigenvalue analysis in ANDES and interpret the results.

In [ ]:
# Reduce logging verbosity for PDF builds
import os
if os.environ.get('SPHINX_BUILD_PDF'):
    import andes
    _orig_config_logger = andes.config_logger
    def _quiet_logger(stream_level=20, *args, **kwargs):
        stream_level = max(stream_level, 30)
        return _orig_config_logger(stream_level, *args, **kwargs)
    andes.config_logger = _quiet_logger

## Setup

In [1]:
%matplotlib inline

import numpy as np
import andes

andes.config_logger(stream_level=20)

## Running Eigenvalue Analysis

Eigenvalue analysis requires an established operating point, so ANDES automatically runs power flow before computing eigenvalues. The simplest way to run eigenvalue analysis is to pass `routine='eig'` to `andes.run()`, which loads the case, runs power flow, and then performs the eigenvalue calculation in a single step.

In [2]:
case_path = andes.get_case('kundur/kundur_full.xlsx')
ss = andes.run(case_path, routine='eig')

Working directory: "/Users/hcui7/repos/andes/docs_new/source/tutorials"


> Loaded config from file "/Users/hcui7/.andes/andes.rc"


> Loaded generated Python code in "/Users/hcui7/.andes/pycode".


Parsing input file "/Users/hcui7/repos/andes/andes/cases/kundur/kundur_full.xlsx"...


Input file parsed in 0.2668 seconds.


...


-> Eigenvalue Analysis:


  Positive       0
  Zeros          1
  Negative      52


Eigenvalue analysis finished in 0.0010 seconds.


Report saved to "kundur_full_eig.txt".


-> Single process finished in 0.4955 seconds.


The output shows that the system has 52 negative eigenvalues, 1 zero eigenvalue, and 0 positive eigenvalues. The zero eigenvalue is expected and corresponds to the rotor angle reference (absolute angles are not observable, only relative angles matter). The absence of positive eigenvalues indicates that the system is stable for small perturbations.

Alternatively, you can run eigenvalue analysis step by step after loading the system:

In [3]:
# Step-by-step approach
ss2 = andes.load(andes.get_case('kundur/kundur_full.xlsx'))
ss2.PFlow.run()
ss2.EIG.run()

Working directory: "/Users/hcui7/repos/andes/docs_new/source/tutorials"


> Loaded config from file "/Users/hcui7/.andes/andes.rc"


> Reloaded generated Python code of module "pycode".


Parsing input file "/Users/hcui7/repos/andes/andes/cases/kundur/kundur_full.xlsx"...


Input file parsed in 0.0185 seconds.


...


-> Eigenvalue Analysis:


  Positive       0
  Zeros          1
  Negative      52


Eigenvalue analysis finished in 0.0009 seconds.


Report saved to "kundur_full_eig.txt".


True

## Plotting Eigenvalues on the S-Plane

The eigenvalue results are most easily interpreted through visualization. The `ss.EIG.plot()` method creates an s-plane plot where the x-axis represents the real part (damping) and the y-axis represents the imaginary part (oscillation frequency in rad/s).

In [4]:
ss.EIG.plot()

/Users/hcui7/repos/andes/andes/routines/eig.py:652: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


(<Figure size 640x480 with 1 Axes>,
 <Axes: xlabel='Real [$s^{-1}$]', ylabel='Imaginary [$s^{-1}$]'>)

In the plot, eigenvalues appearing on the left half of the plane (negative real part) represent stable modes. Those closer to the imaginary axis are less damped and will decay slowly after a disturbance. Eigenvalues with larger imaginary parts correspond to higher-frequency oscillations.

For the Kundur two-area system, you can identify several types of modes:
- Modes near the origin with small imaginary parts are often inter-area oscillation modes
- Modes farther from the origin typically correspond to local generator modes and exciter/governor dynamics
- Real eigenvalues (on the real axis) represent non-oscillatory modes

## Eigenvalue Report

ANDES generates a detailed report file (`*_eig.txt`) containing the computed eigenvalues, their damping ratios, and frequencies. This report is useful for detailed analysis and documentation.

In [5]:
with open('kundur_full_eig.txt', 'r') as f:
    content = f.read()
    # Print first 2000 characters to see the structure
    print(content[:2000])

ANDES 1.9.3.post20+gd85520b62.d20260104
Copyright (C) 2015-2026 Hantao Cui

ANDES comes with ABSOLUTELY NO WARRANTY
Case file: /Users/hcui7/repos/andes/andes/cases/kundur/kundur_full.xlsx
Report time: 01/05/2026 02:45:42 PM

Power flow converged in 5 iterations.
Flat-start: No


EIGENVALUE ANALYSIS REPORT                                    

Positives                          0
Zeros                              1
Negatives                         52

STATISTICS
                     Most Associated              Real             Imag.      Damped Freq.         Frequency       Damping [%]

#1                      LL_x EXDC2 1                -1                 0                 0                 0                 0
#2                      LL_x EXDC2 2                -1                 0                 0                 0                 0
#3                      LL_x EXDC2 3                -1                 0                 0                 0                 0
#4                      

## Parameter Sweeps and Root Loci

One of the most powerful applications of eigenvalue analysis is studying how eigenvalues move as system parameters change. This root locus analysis helps identify parameter ranges that maintain stability and shows which modes are most sensitive to parameter variations.

The `ss.EIG.sweep()` method automates this process by repeatedly computing eigenvalues for different parameter values. In this example, we study how the exciter gain `KA` of the first EXDC2 exciter affects system stability by varying it from 20 to 200.

In [6]:
# Sweep KA from 20 to 200 in 10 steps
ret = ss.EIG.sweep(ss.EXDC2.KA, 1, np.linspace(20, 200, 10))

The `sweep()` method returns a dictionary containing the eigenvalues computed at each parameter value. We can then visualize how specific eigenvalues move using `plot_root_loci()`.

The following plot shows eigenvalues 30 through 50 (zero-indexed). The marker size increases with the parameter value, so larger markers correspond to higher exciter gains. This allows you to track the trajectory of each eigenvalue as the parameter increases.

In [7]:
ss.EIG.plot_root_loci(ret, range(30, 51), left=-3, ymax=4, ymin=-4)

(<Figure size 640x480 with 1 Axes>,
 <Axes: xlabel='Real [$s^{-1}$]', ylabel='Imaginary [$s^{-1}$]'>)

The root locus plot reveals how increasing the exciter gain affects system modes. If eigenvalues move toward the right (toward the imaginary axis or beyond), the system becomes less stable or potentially unstable. If they move left, the system becomes more damped.

This type of analysis is invaluable for tuning controller parameters. By observing the root loci, you can identify the parameter range that provides acceptable damping while avoiding instability.

## Cleanup

In [8]:
!andes misc -C

"/Users/hcui7/repos/andes/docs_new/source/tutorials/kundur_full_out.txt" removed.
"/Users/hcui7/repos/andes/docs_new/source/tutorials/kundur_full_eig.txt" removed.


## Next Steps

- {doc}`08-parameter-sweeps` - Batch simulations for sensitivity studies
- {doc}`09-contingency-analysis` - N-1 contingency screening
- {doc}`10-dynamic-control` - Runtime parameter control